<a href="https://colab.research.google.com/github/rahul9383/News-Category-Classifier/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y tensorflow tf-nightly tensorflow-datasets
!pip install tensorflow tensorflow-datasets
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'msg'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'msg'])

train_df['type'] = train_df['type'].map({'ham': 0, 'spam': 1})
test_df['type'] = test_df['type'].map({'ham': 0, 'spam': 1})


train_labels = train_df['type'].values
train_msg = train_df['msg'].values
test_labels = test_df['type'].values
test_msg = test_df['msg'].values

In [18]:
max_len = 50
vocab_size = 10000

encoder = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=max_len
  )

encoder.adapt(train_msg)


model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy']
)


history = model.fit(
    x=train_msg,
    y=train_labels,
    validation_data=(test_msg, test_labels),
    epochs=10,
    validation_steps=30
)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 99ms/step - accuracy: 0.8600 - loss: 0.5930 - val_accuracy: 0.8657 - val_loss: 0.4038
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - accuracy: 0.8660 - loss: 0.2582 - val_accuracy: 0.8657 - val_loss: 0.1604
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 15s 115ms/step - accuracy: 0.9682 - loss: 0.1216 - val_accuracy: 0.9820 - val_loss: 0.0954
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 98ms/step - accuracy: 0.9916 - loss: 0.0559 - val_accuracy: 0.9885 - val_loss: 0.0525
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 104ms/step - accuracy: 0.9950 - loss: 0.0272 - val_accuracy: 0.9856 - val_loss: 0.0429
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 96ms/step - accuracy: 0.9971 - loss: 0.0165 - val_accuracy: 0.9892 - val_loss: 0.0420
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step - accuracy: 0.9981 - loss: 0.0116 - val_accuracy: 0.9871 - val_loss: 0.0414
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step - accuracy: 0.9990 - loss: 0.0061 

In [20]:
def predict_message(pred_text):

  prediction_prob = model.predict(tf.constant([pred_text]))[0][0]

  if prediction_prob < 0.5:
      label = "ham"
  else:
      label = "spam"

  return (label)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
spam


In [24]:

def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction,ans)
    if prediction!= ans:
      passed = False

  if passed:
    print("Model has Passed the Test Cases")
  else:
    print("Model hasn't Passed the Test Cases")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
ham ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
spam spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
ham ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
spam spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
spam spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
ham ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
ham ham
You passed the challenge. Great job!
